In [ ]:
# !pip install langchain_text_splitters
# !pip install python-dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install langchain_community
# !pip install langchain_text_splitters
# !pip install python-dotenv

In [ ]:
# !pip install huggingface_hub
# !pip install langchain_chroma

In [ ]:
from langchain_community.document_loaders import TextLoader #convert raw text to format langchain can work with
from langchain_text_splitters import CharacterTextSplitter #split into meaningful chunks
from langchain_chroma import Chroma #open source database
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


In [ ]:
#loading dotenv file with keys
load_dotenv()

False

In [ ]:
books = pd.read_csv('/content/drive/MyDrive/books_cleaned.csv')
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


### create database

In [ ]:
#since pandas df cant be read by langchain we need to convert it into text filee

books['tagged_description'].to_csv("tagged_description.txt", sep = "\n" , index = False, header =False)

In [ ]:
# now load them using textloader and then split
raw_documents = TextLoader("tagged_description.txt", encoding="utf8").load()
text_splitter = CharacterTextSplitter(chunk_size= 0, chunk_overlap= 0, separator = '\n')
documents = text_splitter.split_documents(raw_documents)

Streaming output truncated to the last 5000 lines.


In [ ]:
documents[0] #just the description

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import login

# Log in to Hugging Face (if needed)
login(token="")

# Define the model for embeddings
model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"

# Create the Hugging Face embeddings
embedding = HuggingFaceEmbeddings(model_name=model_name)

# Create the Chroma vector store using the Hugging Face embeddings
db_books = Chroma.from_documents(documents,
                                 embedding=embedding)

# Now `db_books` is your vector store, and you can use it for retrieval or other tasks

<ipython-input-10-7a7e90ab892a>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "a book about the universe"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='9781852339463 Many books have described how the universe became the way it is today. But what about the future of the universe? How long might the universe as we recognize it survive? The Future of the Universe takes the reader on a journey through space and time, beginning with a long look at the Earth and solar system, voyaging to the outermost galaxies, and finishing with speculations about the life and fate of the entire universe.'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780141011110 From the bestselling author of The Elegant Universe, Brian Greene's The Fabric of the Cosmos takes us on an irresistible and revelatory journey through the biggest of the big questions. What is reality? Could we exist without space and time? Can we travel to the past? What are the limits of the universe? Brian Greene has made the mysteries of space and time accessible to millions with his acclaimed 

In [ ]:
docs[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9781852339463 Many books have described how the universe became the way it is today. But what about the future of the universe? How long might the universe as we recognize it survive? The Future of the Universe takes the reader on a journey through space and time, beginning with a long look at the Earth and solar system, voyaging to the outermost galaxies, and finishing with speculations about the life and fate of the entire universe.')

In [ ]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
5050,9781852339463,1852339462,The Future of the Universe,A.J. Meadows,Science,NaN,Many books have described how the universe bec...,2007.0,3.75,175.0,4.0,The Future of the Universe,9781852339463 Many books have described how th...


In [ ]:
def retrieve_semantic_recommendations(query: str, top_k: int = 5) -> pd.DataFrame:
  recs = db_books.similarity_search(query, k = top_k)

  books_list = []

  for i in range(len(recs)):
    books_list+= [int(recs[i].page_content.strip('"').split()[0])]


  return books[books["isbn13"].isin(books_list)]

In [ ]:
retrieve_semantic_recommendations("A book about mountains")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3627,9780764586019,0764586017,CliffsNotes on Fitzgerald's The Great Gatsby,Kate Maurer,Literary Criticism,http://books.google.com/books/content?id=gu5aW...,The original CliffsNotes study guides offer a ...,2000.0,3.73,96.0,72.0,CliffsNotes on Fitzgerald's The Great Gatsby,9780764586019 The original CliffsNotes study g...
3635,9780764587276,0764587277,CliffsComplete The Adventures of Huckleberry Finn,Mark Twain,Fiction,http://books.google.com/books/content?id=d7DG3...,"In the CliffsComplete guides, the novel's comp...",2001.0,3.81,264.0,11.0,CliffsComplete The Adventures of Huckleberry Finn,"9780764587276 In the CliffsComplete guides, th..."
4066,9780822002420,0822002426,CliffsNotes on Wright's Black Boy,Carl Senna,Literary Criticism,http://books.google.com/books/content?id=7TRLc...,The original CliffsNotes study guides offer ex...,1971.0,3.75,56.0,3.0,CliffsNotes on Wright's Black Boy,9780822002420 The original CliffsNotes study g...
4070,9780822005339,0822005336,CliffsNotes on Williams' The Glass Menagerie &...,James L. Roberts,Literary Criticism,http://books.google.com/books/content?id=eahFf...,The original CliffsNotes study guides offer a ...,1965.0,3.78,88.0,9.0,CliffsNotes on Williams' The Glass Menagerie &...,9780822005339 The original CliffsNotes study g...
4074,9780822012191,0822012197,CliffsNotes on Faulkner's The Sound and the Fury,James L. Roberts,Literary Criticism,http://books.google.com/books/content?id=fv0xx...,The original CliffsNotes study guides offer ex...,1963.0,3.83,72.0,29.0,CliffsNotes on Faulkner's The Sound and the Fury,9780822012191 The original CliffsNotes study g...


#### Now we will use LLMs to roll up the number of categories we have in our dataset. We will be doing zero shot classification